In [1]:
import glob
import os
import mediapipe as mp
import cv2
import matplotlib.pyplot as plt
import csv
import time

from pathlib import Path

2025-03-09 23:59:57.715743: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-09 23:59:57.743103: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-09 23:59:57.743163: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-09 23:59:57.744157: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-09 23:59:57.748965: I tensorflow/core/platform/cpu_feature_guar

In [2]:
directory = "/home/abhinav/Desktop/AIProjects/General Projects/SLT/sign_recognition/data"
csv_file_path = "/home/abhinav/Desktop/AIProjects/General Projects/SLT/sign_recognition/data/out_data.csv"

In [3]:
# with open(csv_file_path, "w") as f:
#     print("File created")

# f.close()

In [4]:
precision = 3
min_det_conf = 0.2
min_track_conf = 0.2
num_images_to_process = 2
block_data_to_process = 10
sleep_time = 0

In [5]:
def get_short(x, n):
    x = str(x)
    al = x.split(".")
    c = al[0] + "." + al[1][:n]
    return float(c)

In [6]:
start_elements = [i for i in range(0, 300, num_images_to_process)]

In [7]:
start_elements = [start_elements[i:i + block_data_to_process] for i in range(0, len(start_elements), block_data_to_process)]

In [8]:
p = 14
start_element = [start_elements[p]]

In [9]:
# 40 is over
folder = directory + "/41"

c = 0

for start in start_element:
    for k in start:

        data = []

        for image in range(k, k+num_images_to_process):

            image_path = folder + "/" + str(image) + ".jpg"
            # print(image_path)


            data_label = int(folder.split("/")[-1])


            mp_hands = mp.solutions.hands
            hands = mp_hands.Hands(min_detection_confidence=min_det_conf, min_tracking_confidence=min_track_conf)


            # image_path = "/home/abhinav/Desktop/AIProjects/General Projects/SLT/sign_recognition/data/21/0.jpg"
            img = cv2.imread(image_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            # plt.imshow(img)


            results = hands.process(img)
            # print("\n\nlen(results.multi_hand_landmarks)", len(results.multi_hand_landmarks))
            # print("\n\nresults.multi_hand_landmarks", results.multi_hand_landmarks)
            # print("\n\nlen(results.multi_handedness)", len(results.multi_handedness))
            # print("\n\nresults.multi_handedness", results.multi_handedness)
            # print("\n\n\n")

            right_detected = False
            left_detected = False

            left_data = []
            right_data = []

            data_point = []

            if results.multi_handedness is not None:

                for idx, (hand, hand_label) in enumerate(zip(results.multi_hand_landmarks, results.multi_handedness)):
                    # print("Detected: ", idx+1, " hand")
                    # print("Hand classification: ", "Right" if hand_label.classification[0].label == "Left" else "Left")
                    # print("Landmarks: \n")
                    # print(": ", hand.landmark)

                    if hand_label.classification is not None:
                        hand_class = "Right" if hand_label.classification[0].label == "Left" else "Left"

                        if hand_class == "Right":

                            right_detected = True

                            for i in range(21):
                                x1 = get_short(hand.landmark[i].x, precision)
                                y1 = get_short(hand.landmark[i].y, precision)
                                z1 = get_short(hand.landmark[i].z, precision)

                                right_data.extend([x1, y1, z1])
                            

                        else:

                            left_detected = True
                        
                            for i in range(21):
                                x1 = get_short(hand.landmark[i].x, precision)
                                y1 = get_short(hand.landmark[i].y, precision)
                                z1 = get_short(hand.landmark[i].z, precision)

                                left_data.extend([x1, y1, z1])
            del results
            del hands
            del mp_hands
            del img

            if left_data == []:
                left_data = [0 for i in range(63)]
            if right_data == []:
                right_data = [0 for i in range(63)]


            # print("Right data: ", right_data)
            # print("Left data: ", left_data)
            # print("Label: ", data_label)

            if left_data != [] or right_data != []:

                data_point = [image] + left_data + right_data
                data_point.append(data_label)

                # print("Datat point : ", data_point)
                data.append(data_point)
            # break
        
        # Here we get 10 data points
        # write it to csv file and close it
        # del all the variables not used further

        with open(csv_file_path, "a") as f:
            for i in data:
                writer = csv.writer(f)
                writer.writerow(i)
        f.close()
        del data

        print("\n\n\n\n\n\nSleep for some time")
        print("Number of images completed processing : ", k+num_images_to_process)
        time.sleep(sleep_time)
        print("Starting again")

I0000 00:00:1741544999.301411   13960 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741544999.334537   14050 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.01), renderer: NVIDIA GeForce RTX 3050 Laptop GPU/PCIe/SSE2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
/home/abhinav/.local/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
I0000 00:00:1741544999.393599   13960 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741544999.402526   14082 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.01), renderer: NVIDIA GeForce RTX 3050 Laptop GPU/PCIe/SSE2
I0000 00:00:1741544999.454291   13960 gl_context_egl.cc:85] Successfully ini







Sleep for some time
Number of images completed processing :  282
Starting again






Sleep for some time
Number of images completed processing :  284
Starting again


I0000 00:00:1741544999.603754   13960 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741544999.618544   14133 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.01), renderer: NVIDIA GeForce RTX 3050 Laptop GPU/PCIe/SSE2
I0000 00:00:1741544999.684114   13960 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741544999.701341   14150 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.01), renderer: NVIDIA GeForce RTX 3050 Laptop GPU/PCIe/SSE2
I0000 00:00:1741544999.767050   13960 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741544999.779834   14167 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.01), renderer: NVIDIA GeForce RTX 3050 Laptop GPU/PCIe/SSE2








Sleep for some time
Number of images completed processing :  286
Starting again






Sleep for some time
Number of images completed processing :  288
Starting again


I0000 00:00:1741544999.851947   13960 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741544999.880610   14184 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.01), renderer: NVIDIA GeForce RTX 3050 Laptop GPU/PCIe/SSE2
I0000 00:00:1741544999.944270   13960 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741544999.958816   14201 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.01), renderer: NVIDIA GeForce RTX 3050 Laptop GPU/PCIe/SSE2
I0000 00:00:1741545000.018350   13960 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741545000.031705   14218 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.01), renderer: NVIDIA GeForce RTX 3050 Laptop GPU/PCIe/SSE2
I0000 00:00:1741545000.084016   13960 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741545000.098193   14235 gl_context.cc:344] GL version: 3.2 (O

...

I0000 00:00:1741545000.158109   13960 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741545000.171497   14252 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.01), renderer: NVIDIA GeForce RTX 3050 Laptop GPU/PCIe/SSE2
I0000 00:00:1741545000.228541   13960 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741545000.242112   14269 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.01), renderer: NVIDIA GeForce RTX 3050 Laptop GPU/PCIe/SSE2
I0000 00:00:1741545000.296248   13960 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741545000.313425   14286 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.01), renderer: NVIDIA GeForce RTX 3050 Laptop GPU/PCIe/SSE2








Sleep for some time
Number of images completed processing :  294
Starting again






Sleep for some time
Number of images completed processing :  296
Starting again


I0000 00:00:1741545000.382280   13960 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741545000.395798   14303 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.01), renderer: NVIDIA GeForce RTX 3050 Laptop GPU/PCIe/SSE2
I0000 00:00:1741545000.448500   13960 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741545000.463284   14320 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.01), renderer: NVIDIA GeForce RTX 3050 Laptop GPU/PCIe/SSE2
I0000 00:00:1741545000.524811   13960 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741545000.534755   14337 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.01), renderer: NVIDIA GeForce RTX 3050 Laptop GPU/PCIe/SSE2
I0000 00:00:1741545000.587747   13960 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741545000.602988   14354 gl_context.cc:344] GL version: 3.2 (O







Sleep for some time
Number of images completed processing :  298
Starting again






Sleep for some time
Number of images completed processing :  300
Starting again


I0000 00:00:1741545000.662250   13960 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741545000.674373   14371 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.01), renderer: NVIDIA GeForce RTX 3050 Laptop GPU/PCIe/SSE2
I0000 00:00:1741545000.732905   13960 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741545000.746826   14388 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.01), renderer: NVIDIA GeForce RTX 3050 Laptop GPU/PCIe/SSE2


In [10]:
print("Finished processing")

Finished processing


In [11]:
# print("Images processed: ", len(data))
# print("\n#Number of features: 3 for each landmark, 21 landmarks for a hand, 1 for data label")
# print("#In total: (3*(21*2)) + 1 = 127, Let's see how much we get")
# print("\nNumber of features of one data point(one image): ", len(data[0]))

# print("\nFirst 10 data points: ")
# c = 0
# for i in data:
#     if c > 10:
#         break
#     print(i)
#     c += 1

In [12]:
print("Completed ", block_data_to_process, " number of blocks of data")

Completed  10  number of blocks of data
